In [1]:
import time
import torch
import numpy as np
import torch.nn as nn
from torch import optim
from utils import fidelity
from QudiTop.gates import *
from numpy.linalg import norm
from QudiTop.circuit import Circuit
from QudiTop.global_var import DTYPE
from scipy.stats import unitary_group
from QudiTop.expectation import Expectation

np.set_printoptions(linewidth=250)
torch.set_printoptions(linewidth=250)

In [2]:
def ZYZ(d, name, obj, with_phase: bool = False):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit(d, 1)
    index = [[1, 2], [0, 1], [1, 2]]
    for i, ind in enumerate(index):
        str_pr = f'{"".join(str(i) for i in ind)}_{i}'
        circ += RZ(d, ind, f'{name}RZ{str_pr}').on(obj)
        circ += RY(d, ind, f'{name}RY{str_pr}').on(obj)
        circ += RZ(d, ind, f'{name}Rz{str_pr}').on(obj)
        if with_phase:
            circ += GP(d, f'{name}phase_{i}').on(obj)
    return circ


def Cd(d, name, obj, ctrl, state):
    if d != 3:
        raise ValueError('Only works when d = 3')
    circ = Circuit(d, 2)
    circ += RZ(d, [0, 1], f'{name}RZ01').on(obj, ctrl, state)
    circ += RZ(d, [0, 2], f'{name}RZ02').on(obj, ctrl, state)
    circ += GP(d, f'{name}phase').on(obj, ctrl, state)
    return circ


def qutrit_ansatz(gate: UMG, with_phase: bool = True):
    d = gate.dim
    obj = gate.obj_qudits
    name = f'{gate.name}_'
    circ = Circuit(d, 2)
    if len(obj) == 1:
        circ += ZYZ(d, f'{name}', obj[0])
    elif len(obj) == 2:
        circ += ZYZ(d, f'{name}U1_', obj[0])
        circ += Cd(d, f'{name}Cd1_', obj[0], obj[1], 1)
        circ += ZYZ(d, f'{name}U2_', obj[0])
        circ += Cd(d, f'{name}Cd2_', obj[0], obj[1], 2)
        circ += ZYZ(d, f'{name}U3_', obj[0])
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[1], obj[0], 2)
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[1], obj[0], 1)
        circ += RY(d, [1, 2], f'{name}RY1').on(obj[1], obj[0], 0)
        circ += ZYZ(d, f'{name}U4_', obj[0])
        circ += Cd(d, f'{name}Cd3_', obj[0], obj[1], 2)
        circ += ZYZ(d, f'{name}U5_', obj[0])
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[1], obj[0], 2)
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[1], obj[0], 1)
        circ += RY(d, [0, 1], f'{name}RY2').on(obj[1], obj[0], 0)
        circ += ZYZ(d, f'{name}U6_', obj[0])
        circ += Cd(d, f'{name}Cd4_', obj[0], obj[1], 0)
        circ += ZYZ(d, f'{name}U7_', obj[0])
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[1], obj[0], 2)
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[1], obj[0], 1)
        circ += RY(d, [1, 2], f'{name}RY3').on(obj[1], obj[0], 0)
        circ += ZYZ(d, f'{name}U8_', obj[0])
        circ += Cd(d, f'{name}Cd5_', obj[0], obj[1], 2)
        circ += ZYZ(d, f'{name}U9_', obj[0])
    else:
        raise ValueError('Only works when nq <= 2')
    if with_phase:
        circ += [GP(d, 'phase').on(i) for i in obj]
    return circ

In [61]:
d, nq = 3, 2
circ = Circuit(d, nq)
ansatz = Circuit(d, nq)
mat = unitary_group.rvs(d**nq, random_state=42)
obj = list(range(nq))
gate = UMG(d, mat, name=f'mat').on(obj)
circ += gate
ansatz += qutrit_ansatz(gate, True)
for i, g in enumerate(ansatz.gates):
    print('{:3d}'.format(i), g)

pr = ansatz.get_parameters()
g_num = len(ansatz.gates)
p_num = len(pr)
print('Number of qudits: %d' % nq)
print('Number of params: %d' % p_num)
print('Number of gates: %d' % g_num)

psi = circ.get_qs(endian_reverse=True)
rho = np.outer(psi, psi.conj())
print('Hamiltonian Dimension:', rho.shape)
Ham = [(1, UMG(d, rho).on(obj))]
expect = Expectation(Ham)

  0 RZ(3 [1 2] mat_U1_RZ12_0|0)
  1 RY(3 [1 2] mat_U1_RY12_0|0)
  2 RZ(3 [1 2] mat_U1_Rz12_0|0)
  3 RZ(3 [0 1] mat_U1_RZ01_1|0)
  4 RY(3 [0 1] mat_U1_RY01_1|0)
  5 RZ(3 [0 1] mat_U1_Rz01_1|0)
  6 RZ(3 [1 2] mat_U1_RZ12_2|0)
  7 RY(3 [1 2] mat_U1_RY12_2|0)
  8 RZ(3 [1 2] mat_U1_Rz12_2|0)
  9 RZ(3 [0 1] mat_Cd1_RZ01|0 <-: 1 - 1)
 10 RZ(3 [0 2] mat_Cd1_RZ02|0 <-: 1 - 1)
 11 GP(3 mat_Cd1_phase|0 <-: 1 - 1)
 12 RZ(3 [1 2] mat_U2_RZ12_0|0)
 13 RY(3 [1 2] mat_U2_RY12_0|0)
 14 RZ(3 [1 2] mat_U2_Rz12_0|0)
 15 RZ(3 [0 1] mat_U2_RZ01_1|0)
 16 RY(3 [0 1] mat_U2_RY01_1|0)
 17 RZ(3 [0 1] mat_U2_Rz01_1|0)
 18 RZ(3 [1 2] mat_U2_RZ12_2|0)
 19 RY(3 [1 2] mat_U2_RY12_2|0)
 20 RZ(3 [1 2] mat_U2_Rz12_2|0)
 21 RZ(3 [0 1] mat_Cd2_RZ01|0 <-: 1 - 2)
 22 RZ(3 [0 2] mat_Cd2_RZ02|0 <-: 1 - 2)
 23 GP(3 mat_Cd2_phase|0 <-: 1 - 2)
 24 RZ(3 [1 2] mat_U3_RZ12_0|0)
 25 RY(3 [1 2] mat_U3_RY12_0|0)
 26 RZ(3 [1 2] mat_U3_Rz12_0|0)
 27 RZ(3 [0 1] mat_U3_RZ01_1|0)
 28 RY(3 [0 1] mat_U3_RY01_1|0)
 29 RZ(3 [0 1] mat_U3_Rz01_1

In [57]:
psi0 = np.zeros(9)
psi0[0] = 1
mat @ psi0

array([ 0.15954476+0.11470468j,  0.17427048+0.03118134j, -0.29165765-0.07534947j,  0.12067434+0.08272528j,  0.06708706-0.3753798j , -0.23121421-0.31816062j,  0.33115741+0.49783948j, -0.38422199+0.08347445j, -0.01150734+0.07459997j])

In [60]:
psi = circ.get_qs(endian_reverse=True)
psi

array([ 0.15954476+0.11470468j,  0.17427048+0.03118134j, -0.29165765-0.07534947j,  0.12067434+0.08272528j,  0.06708706-0.3753798j , -0.23121421-0.31816062j,  0.33115741+0.49783948j, -0.38422199+0.08347445j, -0.01150734+0.07459997j])

In [66]:
print(mat)

[[ 0.15954476+0.11470468j -0.06103561+0.39748719j  0.35366995+0.0241016j   0.51848236-0.17040294j  0.09154356-0.09661222j  0.00251148+0.12965336j  0.32082762+0.03723391j  0.03906598-0.02118671j -0.47704032-0.08804991j]
 [ 0.17427048+0.03118134j -0.11656296+0.27828594j  0.02472758-0.09278033j  0.12259524-0.21456561j -0.40581082-0.07250741j -0.52954083-0.35859401j -0.43220594+0.06974023j  0.02890947-0.12278077j  0.112041  -0.09092448j]
 [-0.29165765-0.07534947j -0.32234964-0.41684445j  0.02047276-0.1451303j   0.12267488+0.00546674j -0.01424259-0.17246723j -0.29985217+0.00951082j  0.01013678+0.32603134j -0.03580743+0.03965181j -0.316844  +0.51520425j]
 [ 0.12067434+0.08272528j -0.1638045 +0.01528112j -0.02596774-0.42055175j  0.11850007-0.06701917j  0.43053669+0.02593006j  0.06047641+0.35328226j -0.52880273-0.00828177j  0.32873943+0.21876952j  0.05052043-0.05266853j]
 [ 0.06708706-0.3753798j  -0.36879162+0.28566322j -0.37522101+0.10404462j -0.12648422-0.04316762j  0.29023162-0.09856455j -0

In [65]:
print(circ.matrix())

[[[ 0.15954476+0.11470468j -0.06103561+0.39748719j  0.35366995+0.0241016j   0.51848236-0.17040294j  0.09154356-0.09661222j  0.00251148+0.12965336j  0.32082762+0.03723391j  0.03906598-0.02118671j -0.47704032-0.08804991j]
  [ 0.12067434+0.08272528j -0.1638045 +0.01528112j -0.02596774-0.42055175j  0.11850007-0.06701917j  0.43053669+0.02593006j  0.06047641+0.35328226j -0.52880273-0.00828177j  0.32873943+0.21876952j  0.05052043-0.05266853j]
  [ 0.33115741+0.49783948j  0.09622137-0.09261697j  0.05410066+0.13679794j -0.08436478+0.17779801j -0.07660124-0.27261701j  0.05435022+0.31239716j -0.29446256-0.00349759j -0.46440243-0.21108335j -0.09871878+0.1638863j ]]

 [[ 0.17427048+0.03118134j -0.11656296+0.27828594j  0.02472758-0.09278033j  0.12259524-0.21456561j -0.40581082-0.07250741j -0.52954083-0.35859401j -0.43220594+0.06974023j  0.02890947-0.12278077j  0.112041  -0.09092448j]
  [ 0.06708706-0.3753798j  -0.36879162+0.28566322j -0.37522101+0.10404462j -0.12648422-0.04316762j  0.29023162-0.09856

In [62]:
start = time.perf_counter()
p0 = np.random.uniform(-1, 1, p_num)
target = torch.tensor([1], dtype=DTYPE)
ansatz.assign_ansatz_parameters(dict(zip(pr, p0)))
optimizer = optim.Adam(ansatz.parameters(), lr=1e-2)
for i in range(500):
    out = expect(ansatz())
    loss = nn.L1Loss()(out, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        t = time.perf_counter() - start
        print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (loss, out, i, t))
    if loss < 1e-8:
        break
t = time.perf_counter() - start
print('Loss: %.15f, Fidelity: %.15f, %3d, %.4f' % (loss, out, i, t))

pr_res = ansatz.get_parameters()
psi_res = ansatz.get_qs(pr_res)
print('psi norm: %.20f' % norm(psi - psi_res, 2))
print('psi fidelity: %.20f' % fidelity(psi, psi_res))

end = time.perf_counter()
print('Runtime: %f' % (end - start))

Loss: 0.982170080039227, Fidelity: 0.017829919960773,   0, 0.0691
Loss: 0.841614823967881, Fidelity: 0.158385176032119,  10, 0.6698
Loss: 0.803239089873358, Fidelity: 0.196760910126642,  20, 1.2911
Loss: 0.760274777641424, Fidelity: 0.239725222358576,  30, 1.8994
Loss: 0.712912375916501, Fidelity: 0.287087624083499,  40, 2.5144
Loss: 0.649353872617932, Fidelity: 0.350646127382068,  50, 3.1193
Loss: 0.573783395092084, Fidelity: 0.426216604907916,  60, 3.7148
Loss: 0.495815738316319, Fidelity: 0.504184261683681,  70, 4.3178
Loss: 0.416893311329301, Fidelity: 0.583106688670699,  80, 4.9214
Loss: 0.340077910824672, Fidelity: 0.659922089175328,  90, 5.5314
Loss: 0.270000297668807, Fidelity: 0.729999702331193, 100, 6.1426
Loss: 0.209064359545718, Fidelity: 0.790935640454282, 110, 6.7531
Loss: 0.158504669823726, Fidelity: 0.841495330176274, 120, 7.3577
Loss: 0.118033428035369, Fidelity: 0.881966571964631, 130, 8.0478
Loss: 0.086379601227883, Fidelity: 0.913620398772117, 140, 8.7671
Loss: 0.06

In [63]:
psi_res = ansatz.get_qs(pr_res)
print('psi norm: %.20f' % norm(psi - psi_res, 2))
print('psi fidelity: %.20f' % fidelity(psi, psi_res))
expect(ansatz())

psi norm: 1.40697864386785620461
psi fidelity: 0.01294349288212334401


tensor([0.0002], dtype=torch.float64, grad_fn=<StackBackward0>)